In [1]:
# Parameters
api_url = '/services/data/v43.0/sobjects'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4", 
}
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import pandas as pd
import datetime

#stagnant reports
import os
cwd = os.getcwd()

In [3]:
## import all contacts, find all matching contacts, and add the new event venue in

In [3]:
## load input data
import_df = pd.read_csv(cwd+'/Data Files/report1610375760623.csv')
display(import_df.head(5))
venue_name = 'nanoMFG tool users'

,First Name,Last Name,Email,Contact ID,nanoHUB user ID,registration date
0,Zhao,Qin,qinzhao@mit.edu,0035w000034I3mX,77500,2/18/2013
1,Avinash,Lahgere,avinashlahgere@gmail.com,0035w000034ISqK,145868,5/27/2016
2,Alessandro,Pianelli,alessandro.pianelli@wat.edu.pl,0035w000034Jywm,271842,11/20/2019
3,Manikanta,Vemula,ee19acmtech11003@iith.ac.in,0035w000034Isob,278779,2/18/2020
4,SUDHEENDRAN,KOORIYATTIL,sudhi.kooriyattil@gmail.com,0035w00003BRVNe,296143,8/5/2020


In [4]:
mod_df = pd.DataFrame()
mod_df['nanoHUB_user_ID__c'] = import_df['nanoHUB user ID']
display(mod_df.head(5))

,nanoHUB_user_ID__c
0,77500
1,145868
2,271842
3,278779
4,296143


In [5]:
## load in the SF data
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

from DB2SalesforceAPI import DB2SalesforceAPI
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [6]:
sf_df = db_s.query_data('SELECT nanoHUB_user_ID__c, Venue__c FROM Contact')
display(sf_df.head(2))

[Success] Bulk job creation successful. Job ID = 7505w00000TGxLRAA1
{"id":"7505w00000TGxLRAA1","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-11T15:58:16.000+0000","systemModstamp":"2021-01-11T15:58:16.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TGxLRAA1","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-11T15:58:16.000+0000","systemModstamp":"2021-01-11T15:58:17.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000TGxLRAA1","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-11T15:58:16

,Venue__c,nanoHUB_user_ID__c
0,NaN,998.0
1,NaN,1683.0


In [7]:
## find matches
sf_nhids = sf_df['nanoHUB_user_ID__c'].to_list() 
importids = mod_df['nanoHUB_user_ID__c'].to_list()


In [8]:
matches = []
for ind,val in enumerate(sf_nhids): 
    if val in importids:
        matches.append(ind)

In [9]:
len(matches)

876

In [11]:
filtered_df = sf_df.iloc[matches,:]
display(filtered_df.head(2))

,Venue__c,nanoHUB_user_ID__c
943,NaN,44307.0
5668,nanoBIO tool users,45675.0


In [12]:
updated_venue__c = []
for i,j in enumerate(filtered_df['Venue__c'].to_list()):
    try:
        updated_venue__c.append(j+';'+venue_name)
    except:
        updated_venue__c.append(venue_name)


In [13]:
filtered_df['Venue__c'] = updated_venue__c
display(filtered_df.head(2))

C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Venue__c,nanoHUB_user_ID__c
943,nanoMFG tool users,44307.0
5668,nanoBIO tool users;nanoMFG tool users,45675.0


In [14]:
## reset index and upload
filtered_df = filtered_df.reset_index()
display(filtered_df.head(2))

,index,Venue__c,nanoHUB_user_ID__c
0,943,nanoMFG tool users,44307.0
1,5668,nanoBIO tool users;nanoMFG tool users,45675.0


In [15]:
filtered_df = filtered_df.drop(columns = 'index')
display(filtered_df.head(2))

,Venue__c,nanoHUB_user_ID__c
0,nanoMFG tool users,44307.0
1,nanoBIO tool users;nanoMFG tool users,45675.0


In [16]:
db_s.object_id = 'Contact'
db_s.external_id = 'nanoHUB_user_ID__c'
db_s.send_data(filtered_df)

[Success] Bulk job creation successful. Job ID = 7505w00000TGxGrAAL
[Success] CSV upload successful. Job ID = 7505w00000TGxGrAAL
[Success] Closing job successful. Job ID = 7505w00000TGxGrAAL


In [19]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000TGxGrAAL',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-11T16:01:43.000+0000',
 'systemModstamp': '2021-01-11T16:03:28.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 876,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 18420,
 'apiActiveProcessingTime': 17425,
 'apexProcessingTime': 18792}

In [20]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

'"sf__Id","sf__Error",Venue__c,nanoHUB_user_ID__c\n'
